<a href="https://colab.research.google.com/github/yajima-yasutoshi/Model/blob/main/20250730/%E5%B7%A1%E5%9B%9E%E3%82%BB%E3%83%BC%E3%83%AB%E3%82%B9%E3%83%9E%E3%83%B3%E5%95%8F%E9%A1%8C%E3%81%AE%E6%BC%94%E7%BF%92%E5%95%8F%E9%A1%8C%E3%81%AE%E8%A7%A3%E8%AA%AC%E3%81%A8%E8%A7%A3%E7%AD%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習問題 解答 (巡回セールスマン問題)
## 準備

まず、問題を解くために必要なライブラリをインストールする。

In [ ]:
%%capture
# 数理最適化ライブラリ
!pip install mip

# Matplotlibで日本語を表示するためのライブラリをインストール
!pip install japanize-matplotlib

-----

##演習問題1

基本例の都市2の座標を $(4,3)$ から $(5,0)$ に変更し、都市3の座標を $(2,1)$ から $(3,4)$ に変更した場合、最適な巡回路の総移動距離を求めよ。

###解説
都市の座標が変更されたことにより、
都市間の移動コスト（ユークリッド距離）を表すパラメータ $c\_{ij}$ が変化する。

  * **都市の座標:**
      * 都市0: (0, 0) - デポ
      * 都市1: (1, 5)
      * 都市2: (5, 0)  \<-- 変更
      * 都市3: (3, 4)  \<-- 変更

上記座標に基づき、新たな距離行列 $c_{ij}$ を計算し、
基本例と同じMTZ法による定式化を用いて最適化を行う。

###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus
import math

# --- 1. データ定義 ---
# 変更後の都市の座標
cities_coords = {
    0: (0, 0),
    1: (1, 5),
    2: (5, 0),  # 変更後の座標
    3: (3, 4)   # 変更後の座標
}
city_indices = list(cities_coords.keys())
n = len(city_indices) # 都市数

# 距離行列(コスト)の計算
costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        # ユークリッド距離を計算
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# --- 2. モデルの作成 ---
model = Model("TSP_Exercise1")

# --- 3. 決定変数の定義 ---
# x_ij: 都市iから都市jへ移動する場合に1、そうでなければ0
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}

# u_i: 都市iの訪問順序 (デポ0を除く)
u = {i: model.add_var(var_type=INTEGER, lb=2, ub=n, name=f"u_{i}")
     for i in city_indices if i != 0}

# --- 4. 目的関数の設定 ---
# 総移動距離を最小化する
model.objective = minimize(xsum(costs[i, j] * x[i, j] for i, j in x))

# --- 5. 制約条件の設定 ---
# 各都市から必ず1つの都市へ出発する
for i in city_indices:
    model += xsum(x[i, j] for j in city_indices if i != j) == 1, f"leave_{i}"

# 各都市へ必ず1つの都市から到着する
for j in city_indices:
    model += xsum(x[i, j] for i in city_indices if i != j) == 1, f"enter_{j}"

# MTZ法による部分巡回路除去制約
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + n * x[i, j] <= n - 1, f"MTZ_{i}_{j}"

# --- 6. 最適化の実行 ---
status = model.optimize()

# --- 7. 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print(f"総移動距離: {model.objective_value:.4f}")

    # 巡回路の復元
    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in city_indices:
            if current_city != j and x[current_city, j].x >= 0.99:
                tour.append(j)
                current_city = j
                break
    tour.append(0)
    print(f"最適な巡回路: {tour}")
else:
    print("最適解は見つかりませんでした。")

-----

## 演習問題2

基本例の4都市 (0,1,2,3)に、新たに都市4 (座標 (3,6)) を追加する。この5都市 (デポ0を含む)の最適な巡回路を求め、その総移動距離を解答せよ。

### Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus
import math

# --- 1. データ定義 ---
# 新たな都市を追加した座標
cities_coords = {
    0: (0, 0),
    1: (1, 5),
    2: (4, 3),
    3: (2, 1),
    4: (3, 6)  # 追加された都市
}
city_indices = list(cities_coords.keys())
n = len(city_indices) # 都市数 (5になる)

# 距離行列(コスト)の計算
costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# --- 2. モデルの作成 ---
model = Model("TSP_Exercise2")

# --- 3. 決定変数の定義 ---
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
u = {i: model.add_var(var_type=INTEGER, lb=2, ub=n, name=f"u_{i}")
     for i in city_indices if i != 0}

# --- 4. 目的関数の設定 ---
model.objective = minimize(xsum(costs[i, j] * x[i, j] for i, j in x))

# --- 5. 制約条件の設定 ---
for i in city_indices:
    model += xsum(x[i, j] for j in city_indices if i != j) == 1, f"leave_{i}"
for j in city_indices:
    model += xsum(x[i, j] for i in city_indices if i != j) == 1, f"enter_{j}"

# MTZ制約 (n=5)
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + n * x[i, j] <= n - 1, f"MTZ_{i}_{j}"

# --- 6. 最適化の実行 ---
status = model.optimize()

# --- 7. 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print(f"総移動距離: {model.objective_value:.4f}")

    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in city_indices:
            if current_city != j and x[current_city, j].x >= 0.99:
                tour.append(j)
                current_city = j
                break
    tour.append(0)
    print(f"最適な巡回路: {tour}")
else:
    print("最適解は見つかりませんでした。")

-----

## 演習問題3

基本例の4都市問題において、何らかの理由で都市1と都市2の間の直接移動が禁止されたとする。この制約を追加した場合、最適な総移動距離を解答せよ。

###数理モデルの定式化

この問題は、基本例のモデルに新しい制約を追加することで定式化できる。都市1から都市2への移動、および都市2から都市1への移動を禁止する必要がある。これは、対応するバイナリ決定変数 $x_{12}$ と $x_{21}$ を強制的に0にすることで実現できる。

  * **追加される制約条件:**
      * $x\_{12} = 0$
      * $x\_{21} = 0$

これらの制約をモデルに追加することにより、最適化ソルバーはこれらの経路を含まない解の中から最短経路を探索する。

###Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus
import math

# --- 1. データ定義 (基本例と同じ) ---
cities_coords = {
    0: (0, 0), 1: (1, 5), 2: (4, 3), 3: (2, 1)
}
city_indices = list(cities_coords.keys())
n = len(city_indices)

costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# --- 2. モデルの作成 ---
model = Model("TSP_Exercise3")

# --- 3. 決定変数の定義 ---
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
u = {i: model.add_var(var_type=INTEGER, lb=2, ub=n, name=f"u_{i}")
     for i in city_indices if i != 0}

# --- 4. 目的関数の設定 ---
model.objective = minimize(xsum(costs[i, j] * x[i, j] for i, j in x))

# --- 5. 制約条件の設定 ---
# 基本的なTSP制約
for i in city_indices:
    model += xsum(x[i, j] for j in city_indices if i != j) == 1, f"leave_{i}"
for j in city_indices:
    model += xsum(x[i, j] for i in city_indices if i != j) == 1, f"enter_{j}"
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + n * x[i, j] <= n - 1, f"MTZ_{i}_{j}"

# ★★★ 演習問題3の追加制約 ★★★
# 都市1と都市2間の移動を禁止する
model += x[1, 2] == 0, "prohibit_1_to_2"
model += x[2, 1] == 0, "prohibit_2_to_1"

# --- 6. 最適化の実行 ---
status = model.optimize()

# --- 7. 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print(f"総移動距離: {model.objective_value:.4f}")

    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in city_indices:
            if current_city != j and x[current_city, j].x >= 0.99:
                tour.append(j)
                current_city = j
                break
    tour.append(0)
    print(f"最適な巡回路: {tour}")
else:
    print("最適解は見つかりませんでした。")

-----

## 演習問題4

基本例の4都市問題において、都市1を訪問した後(直接でなくてもよい)、必ず都市3を訪問しなければならないという制約を追加せよ。この制約下で最適な巡回路を求め、総移動距離を解答せよ。

### 2\. 数理モデルでの定式化

この問題は、訪問の「順序」に関する制約をモデルに追加する必要がある。「都市1を訪問した後に都市3を訪問する」という条件は、MTZ法で導入した訪問順序を示す補助変数 $u_i$ を用いて簡潔に表現できる。具体的には、都市1の訪問順序 $u_1$ が、都市3の訪問順序 $u_3$ よりも小さい（早い）ことを強制すればよい。

  * **追加される制約条件:**
      * $u_1 \< u_3$

整数変数 $u_i$ を用いているため、この制約は $u_1 \le u_3 - 1$ と等価である。この不等式をモデルに追加する。

### 3\. Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus
import math

# --- 1. データ定義 (基本例と同じ) ---
cities_coords = {
    0: (0, 0), 1: (1, 5), 2: (4, 3), 3: (2, 1)
}
city_indices = list(cities_coords.keys())
n = len(city_indices)

costs = {}
for i in city_indices:
    for j in city_indices:
        if i == j:
            continue
        coord1 = cities_coords[i]
        coord2 = cities_coords[j]
        costs[i, j] = math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

# --- 2. モデルの作成 ---
model = Model("TSP_Exercise4")

# --- 3. 決定変数の定義 ---
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
u = {i: model.add_var(var_type=INTEGER, lb=2, ub=n, name=f"u_{i}")
     for i in city_indices if i != 0}

# --- 4. 目的関数の設定 ---
model.objective = minimize(xsum(costs[i, j] * x[i, j] for i, j in x))

# --- 5. 制約条件の設定 ---
# 基本的なTSP制約
for i in city_indices:
    model += xsum(x[i, j] for j in city_indices if i != j) == 1, f"leave_{i}"
for j in city_indices:
    model += xsum(x[i, j] for i in city_indices if i != j) == 1, f"enter_{j}"
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + n * x[i, j] <= n - 1, f"MTZ_{i}_{j}"

# ★★★ 演習問題4の追加制約 ★★★
# 都市1を訪問した後に都市3を訪問する (u1 < u3)
model += u[1] <= u[3] - 1, "precedence_1_then_3"

# --- 6. 最適化の実行 ---
status = model.optimize()

# --- 7. 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print(f"総移動距離: {model.objective_value:.4f}")

    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in city_indices:
            if current_city != j and x[current_city, j].x >= 0.99:
                tour.append(j)
                current_city = j
                break
    tour.append(0)
    print(f"最適な巡回路: {tour}")
else:
    print("最適解は見つかりませんでした。")

## 演習問題5

基本例の都市間移動コストが、方向によって異なるとする(非対称コスト)。以下のコスト行列 $c\_{ij}$ (都市iから都市jへのコスト)を用いて、最適な巡回路の総移動コストを解答せよ。


$$
c = \begin{pmatrix} \infty & 10 & 15 & 20 \\ 5 & \infty & 9 & 10 \\ 6 & 13 & \infty & 12 \\ 8 & 8 & 9 & \infty \end{pmatrix}
$$


### 2\. 数理モデルの定式化

この問題は**非対称巡回セールスマン問題 (Asymmetric TSP)** と呼ばれる。
これまで扱ってきた対称TSP ($c_{ij} = c_{ji}$) とは異なり、
往路と復路のコストが異なる。
しかし、MTZ法による整数計画モデルは、コスト行列 $c_{ij}$ が対称であるか非対称であるかを問わず、そのまま適用可能である。
変更点は、入力パラメータであるコスト $c_{ij}$ を、ユークリッド距離から計算するのではなく、与えられたコスト行列の値に置き換えることだけである。

### 3\. Python (MIP) による実装

In [ ]:
# 必要なライブラリのインポート
from mip import Model, xsum, minimize, BINARY, INTEGER, OptimizationStatus
import math

# --- 1. データ定義 ---
city_indices = [0, 1, 2, 3]
n = len(city_indices)

# ★★★ 演習問題5の非対称コスト行列 ★★★
# 与えられたコスト行列を辞書形式で定義する
costs = {
    (0, 1): 10, (0, 2): 15, (0, 3): 20,
    (1, 0): 5,  (1, 2): 9,  (1, 3): 10,
    (2, 0): 6,  (2, 1): 13, (2, 3): 12,
    (3, 0): 8,  (3, 1): 8,  (3, 2): 9
}

# --- 2. モデルの作成 ---
model = Model("TSP_Exercise5_Asymmetric")

# --- 3. 決定変数の定義 ---
x = {(i, j): model.add_var(var_type=BINARY, name=f"x_{i}_{j}")
     for i in city_indices for j in city_indices if i != j}
u = {i: model.add_var(var_type=INTEGER, lb=2, ub=n, name=f"u_{i}")
     for i in city_indices if i != 0}

# --- 4. 目的関数の設定 ---
model.objective = minimize(xsum(costs[i, j] * x[i, j] for i, j in costs))

# --- 5. 制約条件の設定 ---
# 基本的なTSP制約
for i in city_indices:
    model += xsum(x[i, j] for j in city_indices if i != j) == 1, f"leave_{i}"
for j in city_indices:
    model += xsum(x[i, j] for i in city_indices if i != j) == 1, f"enter_{j}"
for i in city_indices:
    if i == 0: continue
    for j in city_indices:
        if j == 0 or i == j: continue
        model += u[i] - u[j] + n * x[i, j] <= n - 1, f"MTZ_{i}_{j}"

# --- 6. 最適化の実行 ---
status = model.optimize()

# --- 7. 結果の表示 ---
if status == OptimizationStatus.OPTIMAL or status == OptimizationStatus.FEASIBLE:
    print(f"総移動コスト: {model.objective_value:.4f}")

    tour = [0]
    current_city = 0
    while len(tour) < n:
        for j in city_indices:
            # costs.keys() に存在しない経路は選択されない
            if (current_city, j) in x and x[current_city, j].x >= 0.99:
                tour.append(j)
                current_city = j
                break
    tour.append(0)
    print(f"最適な巡回路: {tour}")
else:
    print("最適解は見つかりませんでした。")